# Loading an X-LoRA model with PEFT, training to come!

In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

from peft import get_peft_model
device = torch.device("cuda")

torch.version.cuda, torch.__version__, torch.backends.cudnn.version(), torch.backends.cudnn.enabled, torch.cuda.is_available()

/home/ericbuehler/miniconda3/envs/py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ericbuehler/.cache/huggingface/token
Login successful


('12.1', '2.4.0+cu121', 90100, True, True)

In [2]:
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

NVIDIA GeForce RTX 4070 Laptop GPU


### Parameters

In [5]:
model_name= 'HuggingFaceH4/zephyr-7b-beta'

bnb_config4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

bnb_config8bit = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="cuda:0",
    torch_dtype=  torch.bfloat16,
    #attn_implementation="flash_attention_2",
)
model.config.use_cache = False
 
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                         #device_map="cuda:0",
                                          device_map="auto",
                                         )
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
tokenizer.add_bos_token, tokenizer.add_eos_token
eos_token= tokenizer.eos_token_id

Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.52it/s]


### Set up X-LoRA model based on pre-trained adaptors

In [ ]:
from peft.tuners.xlora.config import XLoraConfig
from peft.utils.peft_types import PeftType, TaskType

adapters = [
    "lamm-mit/Zephyr_Bioinspired",
    "lamm-mit/Zephyr_CoT",
    "lamm-mit/Zephyr_Chemistry",
    "lamm-mit/Zephyr_Math",
    "lamm-mit/Zephyr_Physics",
    "lamm-mit/Zephyr_Biology",
    "lamm-mit/Zephyr_Mechanics-and-Materials",
    "lamm-mit/Zephyr_Playtupus-Logical",
    "lamm-mit/Zephyr_Protein-Mechanics",
]
adapters = {str(i): file_name for i, file_name in enumerate(adapters)}

peft_config = XLoraConfig(
    task_type=TaskType.CAUSAL_LM,
    peft_type=PeftType.XLORA,
    hidden_size=model.config.hidden_size,
    adapters=adapters,
    xlora_depth=8,
    xlora_size=2048,
    layerwise_scalings=True,
    xlora_dropout_p=0.2,
)
model = get_peft_model(model, peft_config).cuda()

# Training script will come here!